<a href="https://colab.research.google.com/github/nayrr25/Aplicaciones-Financieras-e-IA/blob/main/Semana5_2_Aps_Financieras6_SeriesTiempo_Garch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana5/2_ARMA_y_GARCH.ipynb)

In [1]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.4 MB/s eta 0:00:00


In [2]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from pandas.util._decorators import Appender

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_process import ArmaProcess          # para simular proceso ARMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

from pmdarima.arima import auto_arima
import arch

import matplotlib.pyplot as plt

In [ ]:
diariosg = pd.read_csv('/content/drive/MyDrive/Cruso-ApsFinancieras/semana5/USD_MXN.csv', sep=';')

In [ ]:
# Cargar desde github
# diariosg = pd.read_csv('https://raw.githubusercontent.com/MaxMitre/Aplicaciones-Financieras/main/Semana5/data/USD_MXN.csv', sep=';')

In [ ]:
diariosg

In [ ]:
diariosg.info()

In [ ]:
diariosg = diariosg[['Date', 'Price']].copy()

In [ ]:
diariosg

In [ ]:
type(diariosg.loc[0, 'Date'])

In [ ]:
# Este código nos dará un Warning
diariosg['Date'] = pd.to_datetime(diariosg['Date'], dayfirst=True)

In [ ]:
# Este es el código sin el warning
diariosg.loc[:, 'Date'] = pd.to_datetime(diariosg['Date'], dayfirst=True)

In [ ]:
diariosg

In [ ]:
# Revisar el tipo
type(diariosg['Date'][0])

In [ ]:
diariosg.set_index('Date', drop=True, inplace=True)

In [ ]:
diariosg

In [ ]:
diariosg['Price'].plot(figsize=(10,4), title='Precios Dólar', ylabel = 'Precio',)
plt.show()

In [ ]:
# p-valor para ver si serie es estacionaria
# ¿La serie es estacionaria?
p_value = adfuller(diariosg['Price'])[1]
print(p_value)

In [ ]:
diariosg['price_log'] = np.log(diariosg['Price'])
diariosg['price_log'].plot(figsize=(10,4), title='Transformación logaritmica Precios Dólar', ylabel = 'Precio',)
plt.show()

In [ ]:
diariosg['retornos_log']= diariosg['price_log'].diff()  # en la diferencia perdemos el primer registro
diariosg.head()

In [ ]:
diariosg['retornos_log'][1:].plot(figsize=(10,4), title='Retornos logaritmicos Dólar', ylabel = 'retornos_log')
plt.show()

In [ ]:
plt.rc("figure", figsize=(4,2))
diariosg['price_log'].plot.hist(bins=60)
plt.show()

In [ ]:
plot_acf(diariosg['price_log'], lags=50)
plt.show()

In [ ]:
# COMPARACION plots de retornos_log
diariosg['retornos_log'][1:].plot(figsize=(10,4), title='Retornos logaritmicos Dólar', ylabel = 'retornos_log',)
plt.show()

In [ ]:
plt.rc("figure", figsize=(6,3))
diariosg['retornos_log'][1:].plot.hist(bins=60, title='histogram')
plt.show()

In [ ]:
plot_acf(diariosg['retornos_log'][1:])
plt.show()

In [ ]:
import scipy.stats

In [ ]:
scipy.stats.probplot(diariosg['retornos_log'][1:], plot =  plt)
plt.title("QQ Plot", size = 24)
plt.show()

In [ ]:
# ¿La serie retornos log es estacionaria?
p_value = adfuller(diariosg['retornos_log'][1:])[1]
p_value

# SE rechaza hipotesis nula, siendo aceptada que es estacionaria

In [ ]:
plt.rc("figure", figsize=(6,3))

plot_acf(diariosg['retornos_log'][1:])
plt.show()

plot_pacf(diariosg['retornos_log'][1:])
plt.show()

In [ ]:
# MODELO ARIMA    # REVISAR SI ES FALSE para precios
model_arma = auto_arima(diariosg['retornos_log'][1:],
                        stationary= True,  stepwise=False, maxiter=200, trace=True,
                        information_criterion='aic', max_order=5)
# start_p=2, start_q=2, d=None, max_p=5, max_q=5 por default

In [ ]:
print(model_arma.summary())

In [ ]:
model_arma.predict()

In [ ]:
model_arima = auto_arima(diariosg['price_log'],
                        stationary= False,  stepwise=False, maxiter=200, trace=True,
                        information_criterion='aic',max_order=5)

In [ ]:
print(model_arima.summary())

In [ ]:
# Exploración de residuos del modelo ARMA de Retornos

residuos_arma = pd.Series(model_arma.resid(), index = diariosg.index[1:])
residuos_arma.plot(figsize=(10,4), title='Residuos arma')
plt.show()

In [ ]:
model_arma.plot_diagnostics(figsize=(12,8), lags=10)
plt.show()

In [ ]:
res2 = pow(residuos_arma,2)

res2.plot(figsize=(10,4), title='Residuos al cuadrado')
plt.show()

plot_acf(res2)
plt.show()

plot_pacf(res2)
plt.show()

In [ ]:
### ARCH  (aprox Desviacion estandar de la serie de tiempo)

garch = arch.arch_model(residuos_arma, p=1, q=0, rescale=False)  # VALOR A Q?
model_garch= garch.fit()

In [ ]:
print(model_garch.summary())

In [ ]:
diariosg['volatilidad_condicional']=np.nan
diariosg['volatilidad_condicional'][1:]=model_garch.conditional_volatility
diariosg['volatilidad_condicional'].plot(figsize=(10,4), title='volatilidad condicional')
plt.show()

In [ ]:
# GARCH (ARCH Generalizado)

garch11 = arch.arch_model(residuos_arma, p=1, q=1, rescale=False)
model_garch11= garch11.fit()

print(model_garch11.summary())

In [ ]:
diariosg['volatilidad_condicional']=np.nan
diariosg['volatilidad_condicional'][1:]=model_garch11.conditional_volatility
diariosg['volatilidad_condicional'].plot(figsize=(10,4), title='volatilidad condicional')
plt.show()

# Value at Risk

In [ ]:
# Riesgo del 5% suponiendo una distribución normal

In [ ]:
diariosg.columns

In [ ]:
lim = diariosg['retornos_log'].quantile(0.05)
lim

In [ ]:
diariosg['riesgo_5pc_cte'] = np.nan
diariosg['riesgo_5pc_cte'][1:]=lim

In [ ]:
diariosg.head(10)

In [ ]:
plt.rc("figure", figsize=(6,4))

diariosg['retornos_log'][lambda x:x>=lim].plot.hist(bins=50, title='histogram', color='blue')
diariosg['retornos_log'][lambda x:x<lim].plot.hist(bins=50, title='histogram', color='r')
plt.show()

In [ ]:
from scipy.stats import norm
lim2 = norm(loc = 0 , scale = 1).ppf(0.05)
lim2

In [ ]:
diariosg

In [ ]:
diariosg['retornos_log'][1:].plot(figsize=(15,6), title='Retornos logaritmicos Dólar', ylabel = 'retornos_log')
diariosg['riesgo_5pc_cte'][1:].plot(c='r')
(diariosg['volatilidad_condicional'][1:]*lim2).plot(c='orange')

plt.show()

# Predicción con ARIMA

In [ ]:
n_periods = 20
fc_log, confint_log = model_arima.predict(n_periods=n_periods,  return_conf_int=True, alpha=0.1)
fc, confint = np.exp(fc_log), np.exp(confint_log)

# Vamos a crear una serie para graficar más facilmente
ix_fc = pd.date_range( start='2022-04-05', periods=n_periods, freq='B')  # 2022-04-05 es la fecha final a la que le queremos predecir
fc_series = pd.Series(fc, ix_fc )

lower_series = pd.Series(confint[:, 0],  index=ix_fc )
upper_series = pd.Series(confint[:, 1],  index=ix_fc )

# Plot
plt.rc("figure", figsize=(7,4))
plt.plot(diariosg['Price'])

plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)

plt.title("Predicción del precio con ARIMA")

In [ ]:
# Recordemos el original
diariosg['Price'].plot(figsize=(7,4), title='Precios Dólar', ylabel = 'Precio')
plt.show()

# Predicción Garch

In [ ]:
fc_garch = model_garch.forecast(reindex=False, horizon=n_periods)
fc_garch

**`fc_garch`** es un objeto ARCHModelForcast, entre sus atributos usaremos la media para sumarlo a la predicción de arima.

[documentación](https://arch.readthedocs.io/en/latest/univariate/univariate_volatility_forecasting.html) de la biblioteca arch y el forcasting de volatilidad

In [ ]:
fc_garch.mean

Podemos ver que las predicciones están guardadas en un dataframe